In [1]:
install.packages("tidyverse")
library("tidyverse")
install.packages("Hmisc")
library("Hmisc")
install.packages("pbapply")
library(pbapply)
install.packages("survival")
library(survival)
install.packages("mice")
library(mice)
install.packages("survival")
library(survival)
install.packages("data.table")
library(data.table)
install.packages("lme4")
library(lme4)
install.packages("parallel")
library(parallel)
install.packages("writexl")
library(writexl)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.1     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


Attaching package: ‘Hmisc’


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, units


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/

## XWAS function

In [2]:
XWAS <- function(
    disc_data = NULL, 
    rep_data = NULL, 
    data = NULL,
    multi_imputation = FALSE, #if TRUE provide a list of datasets, if FALSE only a plain dataset
    model_type, 
    exposures, 
    outcome, 
    covariates,
    extra.vars = NULL, 
    extra.covar = NULL,
    replication = TRUE,    #if TRUE provide both disc and rep data, if FALSE only data
    p_adjust = "BH",
    categories = NULL,     #If categories is supplied, it must have columns "Variable" and "Category"
    strata = NULL,
    rand_terms = NULL,
    glm_family = NULL,
    scale = FALSE,
    scale_exclude = NULL,
    cores = NULL,
    interaction = FALSE,
    int.type = NULL,
    int.var = NULL,
    int.option = NULL,
    ...
) {
  
  ### Startup errors ----------------------------------------------------------
  
  `%nin%` <- Negate(`%in%`)
  
  if (replication == TRUE & (missing(disc_data) | missing(rep_data))) {
    stop("both 'disc_data' and 'rep_data' must be supplied")
  } else if (replication == FALSE & missing(data)) {
    stop("'data' must be supplied")
  } else if (multi_imputation == TRUE & replication == TRUE & (inherits(disc_data, "data.frame") | inherits(rep_data, "data.frame"))) {
    stop("must supply a list of dataframes if multi_imputation = TRUE")
  } else if (multi_imputation == TRUE & replication == FALSE & inherits(data, "data.frame")) {
    stop("must supply a list of dataframes if multi_imputation = TRUE")
  } else if (multi_imputation == FALSE & replication == TRUE & (!is.data.frame(disc_data) | !is.data.frame(rep_data))) {
    stop("must supply 'data' as a dataframe")
  } else if (multi_imputation == FALSE & replication == FALSE & !is.data.frame(data)) {
    stop("must supply 'data' as a dataframe")
  } else if (model_type == "coxph" & !grepl("Surv", outcome)) {
    stop("'outcome' must be a Surv object for coxph")
  } else if (!is.null(categories) & (sum(c("Variable", "Category") %nin% colnames(categories)) > 0)) {
    stop("'categories' df must include column headings 'Variable' and 'Category'")
  } else if (p_adjust %nin% c("holm", "hochberg", "hommel", "bonferroni", "BH", "BY", "fdr", "none")) {
    stop("'p_adjust' must be a valid method from p.adjust.methods")
  }
  
  ### Formulas -------------------------------------------------------------------
  
  # message("Initializing XWAS...")
  cat("Initializing XWAS.............")
  flush.console()
  Sys.sleep(1)
  
  if (!is.null(int.var)) {
    covariates <- c(covariates, int.var)
  }
  
  if (model_type == "coxph" & !is.null(strata)) {
    
    covariate_terms <- paste0(covariates, collapse = " + ")
    stratas <- paste0(strata, collapse = ",")
    strata_term <- paste0(paste0("strata(", stratas), ")")
    covariate_terms <- paste(strata_term, covariate_terms, sep = " + ")
    
  } else if (model_type == "lmer" & !is.null(rand_terms)) {
    
    covariate_terms <- paste0(covariates, collapse = " + ")
    random <- paste0(rand_terms, collapse = " + ")
    covariate_terms <- paste(random, covariate_terms, sep = " + ")
    
  } else {
    
    covariate_terms <- paste0(covariates, collapse = " + ")
    
  }
  
  # merge all components together into single formula
  formula <- paste(outcome, covariate_terms, sep = " ~ ")
  
  # set interaction or not
  if (interaction == TRUE) {
    if (int.type == ":") {
      join <- ":"
    } else if (int.type == "*" | is.null(int.type)) {
      join <- "*"
    }
    if (is.null(int.var)) {
      # if no int.var is supplied, use the last covariate listed
      int.var <- covariates[length(covariates)]
    }
  } 
  
  if (interaction == FALSE) {
    join <- " + "
  }
  
  # * coxph ====
  
  # ** Replication ====
  
  if (model_type == "coxph" & replication == TRUE) {
    
    if (multi_imputation == TRUE) {
      
      # discovery
      model_disc <- function(x) {
        if (!is.null(extra.covar) && x %in% extra.vars) {
          models <- lapply(disc_data, function(y)
            survival::coxph(as.formula(
              paste(
                paste(formula, extra.covar, sep = " + "),
                x, sep = join
              )
            ), data = y, ...)
          )
          
          pool <- mice::pool(models)
          rm(models)
          return(pool)
          
        } else {
          models <- lapply(disc_data, function(y)
            survival::coxph(as.formula(
              paste(
                formula,
                x, sep = join
              )
            ), data = y, ...)
          )
          
          pool <- mice::pool(models)
          rm(models)
          return(pool)
        }
      }
      
      # replication
      model_rep <- function(x) {
        if (!is.null(extra.covar) && x %in% extra.vars) {
          models <- lapply(rep_data, function(y)
            survival::coxph(as.formula(
              paste(
                paste(formula, extra.covar, sep = " + "),
                x, sep = join
              )
            ), data = y, ...)
          )
          
          pool <- mice::pool(models)
          rm(models)
          return(pool)
          
        } else {
          models <- lapply(rep_data, function(y)
            survival::coxph(as.formula(
              paste(
                formula,
                x, sep = join
              )
            ), data = y, ...)
          )
          
          pool <- mice::pool(models)
          rm(models)
          return(pool)
        }                    
      }
      
    }  else if (multi_imputation == FALSE) {
      
      # discovery
      model_disc <- function(x) {
        model <- survival::coxph(as.formula(
          paste(
            formula,
            x, sep = join
          )
        ), data = disc_data, ...)
        
        return(model)
      }
      
      # replication
      model_rep <- function(x) {
        model <- survival::coxph(as.formula(
          paste(
            formula,
            x, sep = join
          )
        ), data = rep_data, ...)
        
        return(model)
      }
    }
  } 
  
  #** Non-replication ====
  
  if (model_type == "coxph" & replication == FALSE) {
    
    if (multi_imputation == TRUE) {
      
      # discovery
      model_disc <- function(x) {
        models <- lapply(data, function(y)
          survival::coxph(as.formula(
            paste(
              formula,
              x, sep = join
            )
          ), data = y, ...)
        )
        
        pool <- mice::pool(models)
        rm(models)
        return(pool)
      }
      
    }  else if (multi_imputation == FALSE) {
      
      # discovery
      model_disc <- function(x) {
        model <- survival::coxph(as.formula(
          paste(
            formula,
            x, sep = join
          )
        ), data = data, ...)
        
        return(model)
      }
    }
  }
  
  # * lm ====
  
  # ** Replication ====
  
  if (model_type == "lm" & replication == TRUE) {
    
    if (multi_imputation == TRUE) {
      
      # discovery
      model_disc <- function(x) {
        models <- lapply(disc_data, function(y)
          lm(as.formula(
            paste(
              formula,
              x, sep = join
            )
          ), data = y, ...)
        )
        
        pool <- mice::pool(models)
        rm(models)
        return(pool)
      }
      
      # replication
      model_rep <- function(x) {
        models <- lapply(rep_data, function(y)
          lm(as.formula(
            paste(
              formula,
              x, sep = join
            )
          ), data = y, ...)
        )
        
        pool <- mice::pool(models)
        rm(models)
        return(pool)
      }
      
    }  else if (multi_imputation == FALSE) {
      
      # discovery
      model_disc <- function(x) {
        model <- lm(as.formula(
          paste(
            formula,
            x, sep = join
          )
        ), data = disc_data, ...)
        
        return(model)
      }
      
      # replication
      model_rep <- function(x) {
        model <- lm(as.formula(
          paste(
            formula,
            x, sep = join
          )
        ), data = rep_data, ...)
        
        return(model)
      }
    }
  }
  
  # ** Non-replication ====
  
  if (model_type == "lm" & replication == FALSE) {
    
    if (multi_imputation == TRUE) {
      
      # discovery
      model_disc <- function(x) {
        models <- lapply(data, function(y)
          lm(as.formula(
            paste(
              formula,
              x, sep = join
            )
          ), data = y, ...)
        )
        
        pool <- mice::pool(models)
        rm(models)
        return(pool)
      }
      
    }  else if (multi_imputation == FALSE) {
      
      # discovery
      model_disc <- function(x) {
        model <- lm(as.formula(
          paste(
            formula,
            x, sep = join
          )
        ), data = data, ...)
        
        return(model)
      }
    }
  }
  
  #No exposure-specific extra.covar paths exist in any of the lm branches (MI or non-MI). 
  #If your analysis needs that, it’s an easy extension: add an if (!is.null(extra.covar) && x %in% extra.vars) 
  #wrapper and paste extra.covar into the RHS before adding x, just like in the Cox/MI code.
  
  # * glm ====
  
  # ** Replication ====
  
  if (model_type == "glm" & replication == TRUE) {
    
    if (multi_imputation == TRUE) {
      
      # discovery
      model_disc <- function(x) {
        models <- lapply(disc_data, function(y)
          glm(as.formula(
            paste(
              formula,
              x, sep = join
            )
          ), data = y, family = glm_family, ...)
        )
        
        pool <- mice::pool(models)
        rm(models)
        return(pool)
      }
      
      # replication
      model_rep <- function(x) {
        models <- lapply(rep_data, function(y)
          glm(as.formula(
            paste(
              formula,
              x, sep = join
            )
          ), data = y, family = glm_family, ...)
        )
        
        pool <- mice::pool(models)
        rm(models)
        return(pool)
      }
      
    }  else if (multi_imputation == FALSE) {
      
      # discovery
      model_disc <- function(x) {
        model <- glm(as.formula(
          paste(
            formula,
            x, sep = join
          )
        ), data = disc_data, family = glm_family, ...)
        
        return(model)
      }
      
      # replication
      model_rep <- function(x) {
        model <- glm(as.formula(
          paste(
            formula,
            x, sep = join
          )
        ), data = rep_data, family = glm_family, ...)
        
        return(model)
      }
    }
  }
  
  # ** Non-replication ====
  
  if (model_type == "glm" & replication == FALSE) {
    
    if (multi_imputation == TRUE) {
      
      # discovery
      model_disc <- function(x) {
        models <- lapply(data, function(y)
          glm(as.formula(
            paste(
              formula,
              x, sep = join
            )
          ), data = y, family = glm_family, ...)
        )
        
        pool <- mice::pool(models)
        rm(models)
        return(pool)
      }
      
    }  else if (multi_imputation == FALSE) {
      
      # discovery
      model_disc <- function(x) {
        model <- glm(as.formula(
          paste(
            formula,
            x, sep = join
          )
        ), data = data, family = glm_family, ...)
        
        return(model)
      }
    }
  }
  
  #No exposure-specific extra adjustment here. Unlike the Cox/MI path (which supported extra.covar for selected exposures via extra.vars), 
  #none of the glm branches add extra.covar. If you need “exposure X additionally adjusted for Z”, you’d replicate the Cox/MI pattern.
  
  cat("done", "\n")
  flush.console()
  Sys.sleep(1)
  
  # Scale ----------------------------------------------------------------
  
  if (scale == TRUE) {
    
    cat("Scaling.......................")
    flush.console()
    
    if (!is.null(scale_exclude)) {
      scale_vars <- exposures[which(exposures %nin% scale_exclude)]
    } else {
      scale_vars <- exposures
    }
    
    # scale function
    scale_numeric <- function(x) { 
      # create logical vector of numeric columns
      nums <- sapply(x, is.numeric)
      # only scale exposures
      nums[names(nums) %nin% scale_vars] <- FALSE 
      # perform scale
      x[nums] <- scale(x[nums], scale = TRUE)
      return(x)
    }
    
    if (multi_imputation == TRUE) {
      
      if (replication == TRUE) {
        # scale desired numeric columns
        disc_data <- lapply(disc_data, scale_numeric)
        rep_data <- lapply(rep_data, scale_numeric)
      } else if (replication == FALSE) {
        # scale desired numeric columns
        data <- lapply(data, scale_numeric)
      }
      
    } else if (multi_imputation == FALSE) {
      
      if (replication == TRUE) {
        # scale desired numeric columns
        disc_data <- scale_numeric(disc_data)
        rep_data <- scale_numeric(rep_data)
      } else if (replication == FALSE) {
        # scale desired numeric columns
        data <- scale_numeric(data)
      }
    }
  }
  
  #Even if a covariate is numeric (e.g., age), it won’t be scaled unless it’s included in exposures. 
  #That’s deliberate—keeps covariate effects on their natural scales.
  
  cat("done", "\n")
  flush.console()
  
  # Run XWAS ----------------------------------------------------------------
  
  options(scipen = 999) # turn off scientific notation
  pbapply::pboptions(type = "timer", char = "=") # initialize progress bar
  
  if (replication == TRUE) {
    
    cat("Performing discovery XWAS.....", "\n")
    flush.console()
    
    ## discovery XWAS
    pool_disc <- as.list(seq(1,length(exposures))) # create list to store model results
    pool_disc <- pbapply::pblapply(exposures, model_disc, cl = cores)
    
    cat("Performing replication XWAS...", "\n")
    flush.console()
    
    ## replication XWAS
    pool_rep <- as.list(seq(1,length(exposures))) # create list to store model results
    pool_rep <- pbapply::pblapply(exposures, model_rep, cl = cores)
    
  } else {
    
    cat("Performing XWAS...............", "\n")
    flush.console()
    
    ## XWAS
    pool_disc <- as.list(seq(1,length(exposures))) # create list to store model results
    pool_disc <- pbapply::pblapply(exposures, model_disc, cl = cores)
    
  }
  
  ### Model Summaries -------------------------------------------------------------------
  
  cat("Processing XWAS results.......")
  flush.console()
  
  if (replication == TRUE) {
    
    if (multi_imputation == TRUE) {
      
      # get list of summaries for all pooled models (with beta)
      smry_disc <- lapply(pool_disc,
                          summary,
                          conf.int = TRUE,
                          conf.level = 0.95)
      
      smry_rep <- lapply(pool_rep,
                         summary,
                         conf.int = TRUE,
                         conf.level = 0.95)
      
      # extract all model summaries from the lists and convert to single data frame of estimates
      allvars_disc <- as.data.frame(data.table::rbindlist(smry_disc))
      allvars_rep <- as.data.frame(data.table::rbindlist(smry_rep))
      
    } else if (multi_imputation == FALSE){
      
      # get list of summaries for all pooled models (with beta)
      smry_disc <- lapply(pool_disc,
                          broom::tidy,
                          conf.int = TRUE,
                          conf.level = 0.95)
      
      smry_rep <- lapply(pool_rep,
                         broom::tidy,
                         conf.int = TRUE,
                         conf.level = 0.95)
      
      # extract all model summaries from the lists and convert to single data frame of estimates
      allvars_disc <- as.data.frame(data.table::rbindlist(smry_disc))
      allvars_rep <- as.data.frame(data.table::rbindlist(smry_rep))
    }
    
    # rename column for exposure
    names(allvars_disc)[names(allvars_disc) == "term"] <- "Exposure"
    names(allvars_rep)[names(allvars_rep) == "term"] <- "Exposure"
    
  }
  
  if (replication == FALSE) {
    
    if (multi_imputation == TRUE) {
      
      # get list of summaries for all pooled models (with beta)
      smry_disc <- lapply(pool_disc,
                          summary,
                          conf.int = TRUE,
                          conf.level = 0.95)
      
      # extract all model summaries from the lists and convert to single data frame of estimates
      allvars_disc <- as.data.frame(data.table::rbindlist(smry_disc))
      
    } else if (multi_imputation == FALSE) {
      
      # get list of summaries for all pooled models (with beta)
      smry_disc <- lapply(pool_disc,
                          broom::tidy,
                          conf.int = TRUE,
                          conf.level = 0.95)
      
      # extract all model summaries from the lists and convert to single data frame of estimates
      allvars_disc <- as.data.frame(data.table::rbindlist(smry_disc))
    }
    
    # rename column for exposure
    names(allvars_disc)[names(allvars_disc) == "term"] <- "Exposure"
    
  }
  
  # Exponentiate ------------------------------------------------------------
  
  ### * coxph ====
  if (model_type == "coxph") {
    
    if (replication == TRUE) {
      
      # calculate Hazard Ratio
      allvars_disc$Hazard.Ratio <- exp(allvars_disc$estimate)
      allvars_rep$Hazard.Ratio <- exp(allvars_rep$estimate)
      
      # rename column for betas
      names(allvars_disc)[names(allvars_disc) == "estimate"] <- "Beta"
      names(allvars_rep)[names(allvars_rep) == "estimate"] <- "Beta"
      
      if (multi_imputation == TRUE) {
        
        # exponentiate beta 95% CIs for HR 95% CIs
        allvars_disc$"HR_2.5%" <- exp(allvars_disc$`2.5 %`)
        allvars_disc$"HR_97.5%" <- exp(allvars_disc$`97.5 %`)
        
        allvars_rep$"HR_2.5%" <- exp(allvars_rep$`2.5 %`)
        allvars_rep$"HR_97.5%" <- exp(allvars_rep$`97.5 %`)
        
        # rename column for beta 95% CIs
        names(allvars_disc)[names(allvars_disc) == "2.5 %"] <- "Beta_2.5%"
        names(allvars_disc)[names(allvars_disc) == "97.5 %"] <- "Beta_97.5%"
        
        names(allvars_rep)[names(allvars_rep) == "2.5 %"] <- "Beta_2.5%"
        names(allvars_rep)[names(allvars_rep) == "97.5 %"] <- "Beta_97.5%"
        
      } else if (multi_imputation == FALSE){
        
        # exponentiate beta 95% CIs for HR 95% CIs
        allvars_disc$"HR_2.5%" <- exp(allvars_disc$conf.low)
        allvars_disc$"HR_97.5%" <- exp(allvars_disc$conf.high)
        
        allvars_rep$"HR_2.5%" <- exp(allvars_rep$conf.low)
        allvars_rep$"HR_97.5%" <- exp(allvars_rep$conf.high)
        
        # rename column for beta 95% CIs
        names(allvars_disc)[names(allvars_disc) == "conf.low"] <- "Beta_2.5%"
        names(allvars_disc)[names(allvars_disc) == "conf.high"] <- "Beta_97.5%"
        
        names(allvars_rep)[names(allvars_rep) == "conf.low"] <- "Beta_2.5%"
        names(allvars_rep)[names(allvars_rep) == "conf.high"] <- "Beta_97.5%"
        
      }
    }
    
    if (replication == FALSE) {
      
      # calculate Hazard Ratio
      allvars_disc$Hazard.Ratio <- exp(allvars_disc$estimate)
      
      # rename column for betas
      names(allvars_disc)[names(allvars_disc) == "estimate"] <- "Beta"
      
      if (multi_imputation == TRUE) {
        
        # exponentiate beta 95% CIs for HR 95% CIs
        allvars_disc$"HR_2.5%" <- exp(allvars_disc$`2.5 %`)
        allvars_disc$"HR_97.5%" <- exp(allvars_disc$`97.5 %`)
        
        # rename column for beta 95% CIs
        names(allvars_disc)[names(allvars_disc) == "2.5 %"] <- "Beta_2.5%"
        names(allvars_disc)[names(allvars_disc) == "97.5 %"] <- "Beta_97.5%"
        
      } else if (multi_imputation == FALSE) {
        
        # exponentiate beta 95% CIs for HR 95% CIs
        allvars_disc$"HR_2.5%" <- exp(allvars_disc$conf.low)
        allvars_disc$"HR_97.5%" <- exp(allvars_disc$conf.high)
        
        # rename column for beta 95% CIs
        names(allvars_disc)[names(allvars_disc) == "conf.low"] <- "Beta_2.5%"
        names(allvars_disc)[names(allvars_disc) == "conf.high"] <- "Beta_97.5%"
        
      }
    }
  } 
  
  ### * glm ====
  if (model_type == "glm") {
    
    if (replication == TRUE) {
      
      # calculate Odds Ratio
      allvars_disc$Odds.Ratio <- exp(allvars_disc$estimate)
      allvars_rep$Odds.Ratio <- exp(allvars_rep$estimate)
      
      # rename column for betas
      names(allvars_disc)[names(allvars_disc) == "estimate"] <- "Beta"
      names(allvars_rep)[names(allvars_rep) == "estimate"] <- "Beta"
      
      if (multi_imputation == TRUE) {
        
        # exponentiate beta 95% CIs for HR 95% CIs
        allvars_disc$"OR_2.5%" <- exp(allvars_disc$`2.5 %`)
        allvars_disc$"OR_97.5%" <- exp(allvars_disc$`97.5 %`)
        
        allvars_rep$"OR_2.5%" <- exp(allvars_rep$`2.5 %`)
        allvars_rep$"OR_97.5" <- exp(allvars_rep$`97.5 %`)
        
        # rename column for beta 95% CIs
        names(allvars_disc)[names(allvars_disc) == "2.5 %"] <- "Beta_2.5%"
        names(allvars_disc)[names(allvars_disc) == "97.5 %"] <- "Beta_97.5%"
        
        names(allvars_rep)[names(allvars_rep) == "2.5 %"] <- "Beta_2.5%"
        names(allvars_rep)[names(allvars_rep) == "97.5 %"] <- "Beta_97.5%"
        
      } else if (multi_imputation == FALSE) {
        
        # exponentiate beta 95% CIs for HR 95% CIs
        allvars_disc$"OR_2.5%" <- exp(allvars_disc$conf.low)
        allvars_disc$"OR_97.5%" <- exp(allvars_disc$conf.high)
        
        allvars_rep$"OR_2.5%" <- exp(allvars_rep$conf.low)
        allvars_rep$"OR_97.5%" <- exp(allvars_rep$conf.high)
        
        # rename column for beta 95% CIs
        names(allvars_disc)[names(allvars_disc) == "conf.low"] <- "Beta_2.5%"
        names(allvars_disc)[names(allvars_disc) == "conf.high"] <- "Beta_97.5%"
        
        names(allvars_rep)[names(allvars_rep) == "conf.low"] <- "Beta_2.5%"
        names(allvars_rep)[names(allvars_rep) == "conf.high"] <- "Beta_97.5%"
        
      }
    } 
    
    if (replication == FALSE) {
      
      # calculate Odds Ratio
      allvars_disc$Odds.Ratio <- exp(allvars_disc$estimate)
      
      # rename column for betas
      names(allvars_disc)[names(allvars_disc) == "estimate"] <- "Beta"
      
      if (multi_imputation == TRUE) {
        
        # exponentiate beta 95% CIs for HR 95% CIs
        allvars_disc$"OR_2.5%" <- exp(allvars_disc$`2.5 %`)
        allvars_disc$"OR_97.5%" <- exp(allvars_disc$`97.5 %`)
        
        # rename column for beta 95% CIs
        names(allvars_disc)[names(allvars_disc) == "2.5 %"] <- "Beta_2.5%"
        names(allvars_disc)[names(allvars_disc) == "97.5 %"] <- "Beta_97.5%"
        
      } else if (multi_imputation == FALSE) {
        
        # exponentiate beta 95% CIs for HR 95% CIs
        allvars_disc$"OR_2.5%" <- exp(allvars_disc$conf.low)
        allvars_disc$"OR_97.5%" <- exp(allvars_disc$conf.high)
        
        # rename column for beta 95% CIs
        names(allvars_disc)[names(allvars_disc) == "conf.low"] <- "Beta_2.5%"
        names(allvars_disc)[names(allvars_disc) == "conf.high"] <- "Beta_97.5%"
        
      }
    }
  }
  
  ### * lm ====
  
  if (model_type == "lm") {
    
    if (replication == TRUE) {
      
      # rename column for betas
      names(allvars_disc)[names(allvars_disc) == "estimate"] <- "Beta"
      names(allvars_rep)[names(allvars_rep) == "estimate"] <- "Beta"
      
      if (multi_imputation == TRUE) {
        
        # rename column for beta 95% CIs
        names(allvars_disc)[names(allvars_disc) == "2.5 %"] <- "Beta_2.5%"
        names(allvars_disc)[names(allvars_disc) == "97.5 %"] <- "Beta_97.5%"
        
        names(allvars_rep)[names(allvars_rep) == "2.5 %"] <- "Beta_2.5%"
        names(allvars_rep)[names(allvars_rep) == "97.5 %"] <- "Beta_97.5%"
        
      } else if (multi_imputation == FALSE) {
        
        # rename column for beta 95% CIs
        names(allvars_disc)[names(allvars_disc) == "conf.low"] <- "Beta_2.5%"
        names(allvars_disc)[names(allvars_disc) == "conf.high"] <- "Beta_97.5%"
        
        names(allvars_rep)[names(allvars_rep) == "conf.low"] <- "Beta_2.5%"
        names(allvars_rep)[names(allvars_rep) == "conf.high"] <- "Beta_97.5%"
        
      }
    }
    
    if (replication == FALSE) {
      
      # rename column for betas
      names(allvars_disc)[names(allvars_disc) == "estimate"] <- "Beta"
      
      if (multi_imputation == TRUE) {
        
        # rename column for beta 95% CIs
        names(allvars_disc)[names(allvars_disc) == "2.5 %"] <- "Beta_2.5%"
        names(allvars_disc)[names(allvars_disc) == "97.5 %"] <- "Beta_97.5%"
        
      } else if (multi_imputation == FALSE) {
        
        # rename column for beta 95% CIs
        names(allvars_disc)[names(allvars_disc) == "conf.low"] <- "Beta_2.5%"
        names(allvars_disc)[names(allvars_disc) == "conf.high"] <- "Beta_97.5%"
        
      }
    }
  }
  
  ### Getting accurate p-values  -------------------------------------------------------------------
  
  if (multi_imputation == TRUE) {
    
    ### using summary on a mipo object (output from the pool function) rounds very small p-values to 0
    ### even when we set the # of digits to the max in R (22)
    ### so we have to hard code the math to get the p-values manually from the mipo object
    
    ## discovery 
    tstat <- as.list(seq(1,length(pool_disc))) # create list to store t stats
    p.val <- as.list(seq(1,length(pool_disc))) # create list to store p-values
    p_table <- as.list(seq(1,length(pool_disc))) # create list to p table dfs
    
    # loop to calculate p-values and create table of results
    for (j in seq_along(pool_disc)) {
      # t statistic test using pooled estimate and pooled variance (t)
      tstat[[j]] <- pool_disc[[j]]$pooled$estimate / sqrt(pool_disc[[j]]$pooled$t)
      # calculate p-values
      p.val[[j]] <- as.data.frame(2 * pt(-abs(tstat[[j]]), df = pool_disc[[j]]$pooled$df))
      # create df in p table
      p_table[[j]] <- data.frame(matrix(NA, nrow = nrow(p.val[[j]]), ncol = 2))
      # fill in p table
      p_table[[j]]$X2 <- p.val[[j]][[1]]
      # add in variable names
      p_table[[j]]$X1 <- pool_disc[[j]]$pooled$term
    }
    
    # extract list into a single df
    p_table <- data.table::rbindlist(p_table)
    
    # change column names
    colnames(p_table) <- c("Exposure.p.table",
                           "p.value.full")
    
    # merge p table with XWAS results
    allvars_disc <- cbind(allvars_disc, p_table)
    
    # test to make sure the rows didn't get mixed up
    all.equal(allvars_disc$Exposure, allvars_disc$Exposure.p.table)
    
    # replace all 0 p-values from pooling with actual value
    allvars_disc$p.value <- ifelse(allvars_disc$p.value == 0, 
                                   allvars_disc$p.value.full,
                                   allvars_disc$p.value)
    
    allvars_disc <- subset(allvars_disc, select = -c(Exposure.p.table, p.value.full))
    
  } 
  
  if (multi_imputation == TRUE & replication == TRUE) {
    
    ## replication 
    tstat <- as.list(seq(1,length(pool_rep))) # create list to store t stats
    p.val <- as.list(seq(1,length(pool_rep))) # create list to store p-values
    p_table <- as.list(seq(1,length(pool_rep))) # create list to p table dfs
    
    # loop to calculate p-values and create table of results
    for (j in seq_along(pool_rep)) {
      # t statistic test using pooled estimate and pooled variance (t)
      tstat[[j]] <- pool_rep[[j]]$pooled$estimate / sqrt(pool_rep[[j]]$pooled$t)
      # calculate p-values
      p.val[[j]] <- as.data.frame(2 * pt(-abs(tstat[[j]]), df = pool_rep[[j]]$pooled$df))
      # create df in p table
      p_table[[j]] <- data.frame(matrix(NA, nrow = nrow(p.val[[j]]), ncol = 2))
      # fill in p table
      p_table[[j]]$X2 <- p.val[[j]][[1]]
      # add in variable names
      p_table[[j]]$X1 <- pool_rep[[j]]$pooled$term
    }
    
    # extract list into a single df
    p_table <- data.table::rbindlist(p_table)
    
    # change column names
    colnames(p_table) <- c("Exposure.p.table",
                           "p.value.full")
    
    # merge p table with XWAS results
    allvars_rep <- cbind(allvars_rep, p_table)
    
    # test to make sure the rows didn't get mixed up
    all.equal(allvars_rep$Exposure, allvars_rep$Exposure.p.table)
    
    # replace all 0 p-values with actual value
    allvars_rep$p.value <- ifelse(allvars_rep$p.value == 0, 
                                  allvars_rep$p.value.full,
                                  allvars_rep$p.value)
    
    allvars_rep <- subset(allvars_rep, select = -c(Exposure.p.table, p.value.full))
    
  }
  
  ### Remove covariate estimates ------------------------------------------------
  
  # remove the unwanted rows with coefficients for the covariates from each model
  # strata terms are not listed in output and don't need removing
  
  # add intercept to list of covariates to remove
  # remove_list <- c("intercept", "Intercept", covariates)
  # remove_list <- paste(remove_list, collapse = "|")
  
  # get index of exposures in results column
  exposures_list <- paste(exposures, collapse = "|")
  index_disc <- grepl(exposures_list, allvars_disc$Exposure)
  
  if (replication == TRUE) {
    index_rep <- grepl(exposures_list, allvars_rep$Exposure)
    if (interaction == TRUE) {
      if (int.option == "both" | is.null(int.option)) {
        allvars_disc <- allvars_disc[index_disc, ]
        allvars_rep <- allvars_rep[index_rep, ]
      } else if (int.option == "int.only") {
        index2_disc <- which(grepl(int.var, allvars_disc$Exposure) & grepl(exposures_list, allvars_disc$Exposure))
        index2_rep <- which(grepl(int.var, allvars_rep$Exposure) & grepl(exposures_list, allvars_rep$Exposure))
        allvars_disc <- allvars_disc[index2_disc, ]
        allvars_rep <- allvars_rep[index2_rep, ]
      }
    } else if (interaction == FALSE) {
      allvars_disc <- allvars_disc[index_disc, ]
      allvars_rep <- allvars_rep[index_rep, ]
    }
  }
  
  if (replication == FALSE) {
    if (interaction == TRUE) {
      index2_disc <- which(grepl(int.var, allvars_disc$Exposure) & grepl(exposures_list, allvars_disc$Exposure))
      if (int.option == "both" | is.null(int.option)) {
        allvars_disc <- allvars_disc[index_disc, ]
      } else if (int.option == "int.only") {
        allvars_disc <- allvars_disc[index2_disc, ]
      }
    } else if (interaction == FALSE) {
      allvars_disc <- allvars_disc[index_disc, ]
    }
  }
  
  ### P-value replication -------------------------------------------------------------------
  
  if (replication == TRUE) {
    
    # get p-value column from replication set to merge
    rep <- subset(allvars_rep, select = c("Exposure", "p.value")) # subset replication results to just p-values
    names(rep)[names(rep) == "p.value"] <- "p.value.rep" # re-name p-values to differentiate from discovery
    
    ## merge disc and rep, keeping estimates just from discovery
    XWAS_total <- merge(allvars_disc, 
                        rep, 
                        by = "Exposure", 
                        all = FALSE, 
                        sort = FALSE)
    
  } else {
    
    ## merge disc and rep, keeping estimates just from discovery
    XWAS_total <- allvars_disc
    
  }
  
  #Because it’s an inner join, any exposure term that failed to appear in replication 
  #(e.g., dropped due to collinearity, zero variance, separation) will be dropped entirely from XWAS_total. 
  #If you wanted to keep discovery-only results for auditing, you could switch to:
  #merge(allvars_disc, rep, by = "Exposure", all.x = TRUE, sort = FALSE)
  
  ### Add variable names and categories -------------------------------------------------------------------
  
  ## map variable names to results
  XWAS_total$Variable <- NA
  # must sort for variable matching to work properly
  variables <- sort(exposures)
  variables_grep <- paste0("\\b", variables)
  
  # (e.g., match "alcohol_freq" to "alcohol_freqDaily or almost daily")
  for(j in seq_along(variables)) {
    XWAS_total$Variable[grepl(variables_grep[[j]], XWAS_total$Exposure)] <- variables[[j]]
  }
  
  if (!is.null(categories)) {
    ## add in categories according to match with variable name
    XWAS_total <- merge(XWAS_total,  
                        subset(categories, select = c("Variable", "Category")), 
                        by = "Variable", 
                        all = FALSE, 
                        sort = FALSE)
    
    # order exposures by category
    XWAS_total <- XWAS_total[order(XWAS_total$Category), ] 
  }
  
  ### Multiple testing correction -------------------------------------------------------------------
  
  if (p_adjust != "none") {
    
    if (replication == TRUE) {
      
      ## calculate FDR in the discovery analyses using benjamini-hochberg method
      XWAS_total$FDR.disc <- p.adjust(XWAS_total$p.value, method = p_adjust)
      
      ## calculate FDR in the replication analyses only among those from the discovery with FDR p-value < 0.05
      exposome_rep <- subset(XWAS_total, select = c("Exposure", 
                                                    "FDR.disc",
                                                    "p.value.rep"))
      
      # subset to associations that were significant after FDR in discovery
      exposome_rep <- exposome_rep[which(exposome_rep$FDR.disc < 0.05), ] 
      
      # calculate FDR on replication p-values only for significant variables in discovery
      exposome_rep$FDR.rep <- p.adjust(exposome_rep$p.value.rep, method = p_adjust) 
      
      # subset replication results to just FDR p-values
      sig <- subset(exposome_rep, select = c("Exposure", "FDR.rep")) 
      
      ## recombine FDR p-values from replication with discovery results
      XWAS_total <- merge(XWAS_total, 
                          sig, 
                          by = "Exposure", 
                          all = T, 
                          sort = F)
      
    } else if (replication == FALSE) {
      ## calculate FDR in the discovery analyses using benjamini-hochberg method
      XWAS_total$FDR <- p.adjust(XWAS_total$p.value, method = p_adjust)
    } 
  }
  
  ### Order columns -------------------------------------------------------------------
  
  reorder <- function(x, y){
    vec <- c()
    for (i in 1:length(x)){
      vec <- c(vec, which(grepl(y[i],names(x))))
    }
    return(x[,vec])
  }
  
  cols <- c("Exposure", "Beta", "Hazard.Ratio", "HR", "Odds.Ratio", "OR", "p.value", "FDR", "Variable", "Category", "std.error", "statistic", "df")
  XWAS_total <- reorder(XWAS_total, y = cols)
  
  # Return ------------------------------------------------------------------
  
  formula <- paste(formula, "x", sep = join)
  
  if (replication == TRUE) {
    
    names(pool_disc) <- exposures
    names(pool_rep) <- exposures
    
    if (multi_imputation == TRUE) {
      
      exposome_results <- list(
        "pooled_models_discovery" = pool_disc,
        "pooled_models_replication" = pool_rep,
        "summary" = XWAS_total,
        "model_formula" = formula,
        "discovery_model_function" = model_disc,
        "replication_model_function" = model_rep,
        "exposures" = exposures
      )
      
      class(exposome_results) <- "XWAS"
      return(exposome_results) 
      
    } else if (multi_imputation == FALSE) {
      
      exposome_results <- list(
        "models_discovery" = pool_disc,
        "models_replication" = pool_rep,
        "summary" = XWAS_total,
        "model_formula" = formula,
        "discovery_model_function" = model_disc,
        "replication_model_function" = model_rep,
        "exposures" = exposures
      )
      
      class(exposome_results) <- "XWAS"
      return(exposome_results) 
    }
    
  }
  
  if (replication == FALSE) {
    
    names(pool_disc) <- exposures
    
    if (multi_imputation == TRUE) {
      
      exposome_results <- list(
        "pooled_models" = pool_disc,
        "summary" = XWAS_total,
        "model_formula" = formula,
        "model_function" = model_disc,
        "exposures" = exposures
      )
      
      class(exposome_results) <- "XWAS"
      return(exposome_results) 
      
    } else if (multi_imputation == FALSE) {
      
      exposome_results <- list(
        "models" = pool_disc,
        "summary" = XWAS_total,
        "model_formula" = formula,
        "model_function" = model_disc,
        "exposures" = exposures
      )
      
      class(exposome_results) <- "XWAS"
      return(exposome_results) 
    }
  }
  
  cat("done")
  flush.console()
}

#Tiny nits (FYI) from ChatGPT

#The code earlier had small label typos (e.g., "HR_97.5" missing % in replication branches, 
#and “HR” comments in the glm section). Purely cosmetic, but worth fixing for clean columns.
#In the non-MI Cox branches, extra.covar for selected exposures wasn’t added (it was added only in the MI Cox branch). 
#If you need exposure-specific extra adjustment without MI, mirror that logic there too.
#If you want, tell me your outcome type (cox/lm/glm), whether you’ll use MI and replication, 
#and I’ll sketch a minimal call + show which fields you’ll read from the returned object.




## Import dataset

In [3]:
system('dx download project-Gzb5YJQJgQzb41qJj9G5G1GV:###YOUR PATH/LVmyo_motion_embeddings_32_VAE_NS_78113.csv')
system('dx download project-Gzb5YJQJgQzb41qJj9G5G1GV:###YOUR PATH/img_features_all_rf_imputed_81k_with_biochemistry_18545_ethnicity.csv')
system('dx download project-Gzb5YJQJgQzb41qJj9G5G1GV:###YOUR PATH/bridge_18545_40616_47602.txt')
system('dx download ###YOUR PATH/Exposome_imputed_recoded_pna.rds')

In [ ]:
bridge  <- read.csv("bridge_18545_40616_47602.txt", sep = "\t", stringsAsFactors = FALSE)
exp_raw <- readRDS("Exposome_imputed_recoded_pna.rds")
feature_raw <- read.csv("img_features_all_rf_imputed_81k_with_biochemistry_18545_ethnicity.csv")
latent  <- read.csv("LVmyo_motion_embeddings_32_VAE_NS_78113.csv")


In [95]:
# ---- keep keys! ----
exp <- exp_raw |>
  select(-menopause_age, -sex, -adopted) |>
  rename(eid_40616 = eid) |>
  mutate(
      NO2_2005_2010 = rowSums(across(c(NO2_2005, NO2_2006, NO2_2007, NO2_2010)), na.rm = TRUE)
  ) |>  
  select(
      eid_40616,
    #eid_40616, country_birth, breastfed, body_size_10yrs_old, height_10yrs_old, handedness, part_multiple_birth, maternal_smoking, birth_weight, family_visit_freq, confide_freq, loneliness, accommodation_type, own_or_rent, years_at_address, hshld_number, hshld_income, neuroticism, mood_swings, miserableness, irritability, sensitivity, fed_up_feelings, nervous, worrier, tense, worry_embarassment, nerves, guilty_feelings, risk_taking, depressed_mood_freq, unenthusiasm_freq, tenseness_freq, tiredness_freq, mobile_phone_weekly_usage, speakerphone, computer_games, easy_wake, chronotype, nap, sleep_difficulty, snoring, ever_smoked, pack_years_prop, pack_years, smoking_status, 
    #tobacco, alcohol_freq, oily_fish, non_oily_fish, processed_meat, poultry, cheese, salt, tea, coffee, water, diet_change_5yrs, diet_variation, summer_outdoors_time, winter_outdoors_time, skin_tan_ease, childhood_sunburn_number, sun_protection_use, solarium_freq, same_sex_intercourse, IPAQ_activity_group, townsend_deprivation_index, close_to_major_road, inverse_distance_nearest_major_road, inverse_distance_nearest_road, NO2_2005, NO2_2006, NO2_2007, NO2_2010, NO_2010, PM10_2007, PM10_2010, PM2.5_absorbance_2010, PM2.5_2010, PM2.5_to_10_2010, road_length_sum_100m,
    #traffic_load_major_roads, traffic_intensity_nearest_major_road, traffic_intensity_nearest_road, sound_average_16hr, sound_average_24hr, daytime_sound_average, evening_sound_average, night_sound_average, natural_environment_buffer_1000m, natural_environment_buffer_300m, greenspace_buffer_1000m, greenspace_buffer_300m, domestic_garden_buffer_1000m, domestic_garden_buffer_300m, water_buffer_1000m, water_buffer_300m, distance_to_coast, population_density, pleasure_walks_freq_4wks, heavy_DIY_4wks, light_DIY_4wks, other_exercise_4wks, strenuous_sports_4wks, death_relative, death_partner, financial_difficulty,
    #divorce, adult_education, pub, religious_group, gym, hshld_grandchild, hshld_partner, hshld_children, gas_fire_heat, gas_hob_heat, open_fire_heat, volunteer, student, employed, home_maker, retired, unemployed, calcium_supplements, fish_oil_supplements, glucosamine_supplements, iron_supplements, selenium_supplements, zinc_supplements, folate_supplements, multivitamin_supplements, vitamin_A_supplements, vitamin_B_supplements, vitamin_C_supplements, vitamin_D_supplements, vitamin_E_supplements, sleep_hours_categorical, bread_fiber, cereal_fiber, total_fruit, total_veg, red_meat, OPA, sedentary  
    alcohol_freq, processed_meat, red_meat, coffee, cheese, mobile_phone_weekly_usage, own_or_rent, PM2.5_2010, NO_2010, sleep_difficulty, snoring, sleep_hours_categorical, pack_years, smoking_status, financial_difficulty, hshld_income, NO2_2005_2010               
  ) 
  #  eid_40616,                                   
  #  hshld_income, depressed_mood_freq, snoring, smoking_status, alcohol_freq,
  #  townsend_deprivation_index, NO_2010, PM2.5_2010, sleep_hours_categorical,
  #  bread_fiber, cereal_fiber, total_fruit, total_veg, red_meat, OPA, sedentary
  #)

feature <- feature_raw |>
  select(
    eid_18545,                                   
    age_at_MRI, Sex, BSA, Ethnicity, LVEF, LVEDVi, LVESVi, Ecc_Global, Err_Global, Ell_Global, PDSR_longit, PDSR_radial, PDSR_circum  
  )



# ---- build CMR table: latent x feature on eid_18545 ----
cmr <- latent |>
  inner_join(feature, by = "eid_18545")

# ---- map exp (40616) -> 18545 via bridge, then join to cmr ----
exp_linked <- exp |>
  left_join(bridge[, c("eid_40616", "eid_18545")], by = "eid_40616")

xwas_data <- exp_linked |>
  inner_join(cmr, by = "eid_18545")  |>
  select(-eid_18545)
  


In [96]:
data_dictionary <- tribble(
  ~Variable, ~Category, ~Field,
  "country_birth", "Early life factors", "1647",
  "breastfed", "Early life factors", "1677",
  "body_size_10yrs_old", "Early life factors", "1687",
  "height_10yrs_old", "Early life factors", "1697",
  "handedness", "Early life factors", "1707",
  "part_multiple_birth", "Early life factors", "1777",
  "maternal_smoking", "Early life factors", "1787",
  "birth_weight", "Early life factors", "20022",
  "family_visit_freq", "Social support", "1031",
  "confide_freq", "Social support", "2110",
  "loneliness", "Social support", "2020",
  "accommodation_type", "Household", "670",
  "own_or_rent", "Household", "680",
  "years_at_address", "Household", "699",
  "hshld_number", "Household", "709",
  "hshld_income", "Household", "738",
  "neuroticism", "Mental health", "20127",
  "mood_swings", "Mental health", "1920",
  "miserableness", "Mental health", "1930",
  "irritability", "Mental health", "1940",
  "sensitivity", "Mental health", "1950",
  "fed_up_feelings", "Mental health", "1960",
  "nervous", "Mental health", "1970",
  "worrier", "Mental health", "1980",
  "tense", "Mental health", "1990",
  "worry_embarassment", "Mental health", "2000",
  "nerves", "Mental health", "2010",
  "guilty_feelings", "Mental health", "2030",
  "risk_taking", "Mental health", "2040",
  "depressed_mood_freq", "Mental health", "2050",
  "unenthusiasm_freq", "Mental health", "2060",
  "tenseness_freq", "Mental health", "2070",
  "tiredness_freq", "Mental health", "2080",
  "mobile_phone_weekly_usage", "Electronics use", "1120",
  "speakerphone", "Electronics use", "1130",
  "computer_games", "Electronics use", "2237",
  "easy_wake", "Sleep", "1170",
  "chronotype", "Sleep", "1180",
  "nap", "Sleep", "1190",
  "sleep_difficulty", "Sleep", "1200",
  "snoring", "Sleep", "1210",
  "ever_smoked", "Smoking", "20160",
  "pack_years_prop", "Smoking", "20162",
  "pack_years", "Smoking", "20161",
  "smoking_status", "Smoking", "20116",
  "tobacco", "Smoking", "1239",
  "alcohol_freq", "Alcohol", "1558",
  "oily_fish", "Diet", "1329",
  "non_oily_fish", "Diet", "1339",
  "processed_meat", "Diet", "1349",
  "poultry", "Diet", "1359",
  "cheese", "Diet", "1408",
  "salt", "Diet", "1478",
  "tea", "Diet", "1488",
  "coffee", "Diet", "1498",
  "water", "Diet", "1528",
  "bread_fiber", "Diet", "derived",  
  "cereal_fiber", "Diet", "derived", 
  "total_fruit", "Diet", "derived", 
  "total_veg", "Diet", "derived", 
  "red_meat", "Diet", "derived",   
  "diet_change_5yrs", "Diet", "1538",
  "diet_variation", "Diet", "1548",
  "summer_outdoors_time", "Sun exposure", "1050",
  "winter_outdoors_time", "Sun exposure", "1060",
  "skin_tan_ease", "Sun exposure", "1727",
  "childhood_sunburn_number", "Sun exposure", "1737",
  "sun_protection_use", "Sun exposure", "2267",
  "solarium_freq", "Sun exposure", "2277",
  "same_sex_intercourse", "Sexual history", "2159",
  "IPAQ_activity_group", "Physical activity", "22032",
  "pleasure_walks_freq_4wks", "Physical activity", "6164", 
  "heavy_DIY_4wks", "Physical activity", "6164",  
  "light_DIY_4wks", "Physical activity", "6164",   
  "other_exercise_4wks", "Physical activity", "6164", 
  "strenuous_sports_4wks", "Physical activity", "6164",  
  "townsend_deprivation_index", "Material deprivation", "189",
  "close_to_major_road", "Physical environment", "24014",
  "inverse_distance_nearest_major_road", "Physical environment", "24012",
  "inverse_distance_nearest_road", "Physical environment", "24010",
  "NO2_2005", "Physical environment", "24016",
  "NO2_2006", "Physical environment", "24017",
  "NO2_2007", "Physical environment", "24018",
  "NO2_2010", "Physical environment", "24003",
  "NO_2010", "Physical environment", "24004",
  "NO2_2005_2010", "Physical environment", "derived",  
  "PM10_2007", "Physical environment", "24019",
  "PM10_2010", "Physical environment", "24005",
  "PM2.5_absorbance_2010", "Physical environment", "24007",
  "PM2.5_2010", "Physical environment", "24006",
  "PM2.5_to_10_2010", "Physical environment", "24008",
  "sound_average_16hr", "Physical environment", "24023",
  "sound_average_24hr", "Physical environment", "24024",
  "daytime_sound_average", "Physical environment", "24020",
  "evening_sound_average", "Physical environment", "24021",
  "night_sound_average", "Physical environment", "24022",
  "natural_environment_buffer_1000m", "Physical environment", "24506",
  "natural_environment_buffer_300m", "Physical environment", "24507",
  "greenspace_buffer_1000m", "Physical environment", "24500",
  "greenspace_buffer_300m", "Physical environment", "24503",
  "domestic_garden_buffer_1000m", "Physical environment", "24501",
  "domestic_garden_buffer_300m", "Physical environment", "24504",
  "water_buffer_1000m", "Physical environment", "24502",
  "water_buffer_300m", "Physical environment", "24505",
  "distance_to_coast", "Physical environment", "24508",
  "population_density", "Physical environment", "20118",
  "road_length_sum_100m", "Physical environment", "24015",
  "traffic_load_major_roads", "Physical environment", "24013",
  "traffic_intensity_nearest_major_road", "Physical environment", "24011",
  "traffic_intensity_nearest_road", "Physical environment", "24009",
  "death_relative", "Stressful life events", "6147",
  "death_partner", "Stressful life events", "6148",
  "financial_difficulty", "Stressful life events", "6150",
  "divorce", "Stressful life events", "6149",
  "adult_education", "Social support", "6163",
  "pub", "Social support", "6161",
  "religious_group", "Social support", "6162",
  "gym", "Physical activity", "6160",
  "OPA", "Physical activity", "derived",
  "sedentary", "Physical activity", "derived",  
  "hshld_grandchild", "Household", "6141",
  "hshld_partner", "Household", "6141",
  "hshld_children", "Household", "6141",
  "gas_fire_heat", "Household", "6139",
  "gas_hob_heat", "Household", "6139",
  "open_fire_heat", "Household", "6139",
  "gas_central_heat", "Household", NA,
  "electric_storage_heat", "Household", NA,
  "oil_central_heat", "Household", NA,
  "volunteer", "Employment", "6160",
  "student", "Employment", "6142",
  "employed", "Employment", "6142",
  "home_maker", "Employment", "6142",
  "retired", "Employment", "6142",
  "unemployed", "Employment", "6142",
  "calcium_supplements", "Supplements", "6179",
  "fish_oil_supplements", "Supplements", "6179",
  "glucosamine_supplements", "Supplements", "6179",
  "iron_supplements", "Supplements", "6179",
  "selenium_supplements", "Supplements", "6179",
  "zinc_supplements", "Supplements", "6179",
  "folate_supplements", "Supplements", "6155",
  "multivitamin_supplements", "Supplements", "6155",
  "vitamin_A_supplements", "Supplements", "6155",
  "vitamin_B_supplements", "Supplements", "6155",
  "vitamin_C_supplements", "Supplements", "6155",
  "vitamin_D_supplements", "Supplements", "6155",
  "vitamin_E_supplements", "Supplements", "6155",
  "sleep_hours_categorical", "Sleep", "derived"
)

In [97]:
set_contrasts <- function(x) {

  # find ordered variables
  ord <- sapply(x, is.ordered)

  # set contrasts to only linear
  for (k in seq(ncol(x[ord]))) {
    contrasts(x[ord][[k]], how.many = 1) <- contr.poly(nlevels(x[ord][[k]]))
  }

  return(x)

}

xwas_data <- set_contrasts(xwas_data)

In [ ]:
types <- tibble(
  variable   = names(xwas_data),
  class      = map_chr(xwas_data, ~ class(.x)[1]),
  is_factor  = map_lgl(xwas_data, is.factor),
  is_numeric = map_lgl(xwas_data, is.numeric)
)
factor_vars <- types |> 
  filter(is_factor) |> 
  pull(variable)  


In [99]:
exposures <- setdiff(names(xwas_data), non_exposure_cols)


In [133]:
#test with covariates, categories and strata
outcome     <- "z_1"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_1 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

Initializing XWAS.............done 
Scaling.......................done 
Performing XWAS............... 
  |==================================================| 100% elapsed=01s  
Processing XWAS results.......

In [ ]:
#test with covariates, categories and strata
outcome     <- "z_2"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_2 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

In [ ]:
#test with covariates, categories and strata
outcome     <- "z_3"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_3 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

In [ ]:
#test with covariates, categories and strata
outcome     <- "z_4"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_4 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

In [ ]:
#test with covariates, categories and strata
outcome     <- "z_5"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_5 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

In [ ]:
#test with covariates, categories and strata
outcome     <- "z_6"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_6 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

In [ ]:
#test with covariates, categories and strata
outcome     <- "z_7"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_7 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

In [ ]:
#test with covariates, categories and strata
outcome     <- "z_8"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_8 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

In [ ]:
#test with covariates, categories and strata
outcome     <- "z_9"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_9 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

In [ ]:
#test with covariates, categories and strata
outcome     <- "z_10"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_10 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

In [ ]:
#test with covariates, categories and strata
outcome     <- "z_11"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_11 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

In [ ]:
#test with covariates, categories and strata
outcome     <- "z_12"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_12 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

In [ ]:
#test with covariates, categories and strata
outcome     <- "z_13"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_13 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

In [ ]:
#test with covariates, categories and strata
outcome     <- "z_14"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_14 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

In [ ]:
#test with covariates, categories and strata
outcome     <- "z_15"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_15 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

In [ ]:
#test with covariates, categories and strata
outcome     <- "z_16"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_16 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

In [ ]:
#test with covariates, categories and strata
outcome     <- "z_17"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_17 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

In [ ]:
#test with covariates, categories and strata
outcome     <- "z_18"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_18 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

In [ ]:
#test with covariates, categories and strata
outcome     <- "z_19"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_19 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

In [ ]:
#test with covariates, categories and strata
outcome     <- "z_20"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_20 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

In [ ]:
#test with covariates, categories and strata
outcome     <- "z_21"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_21 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

In [ ]:
#test with covariates, categories and strata
outcome     <- "z_22"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_22 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

In [ ]:
#test with covariates, categories and strata
outcome     <- "z_23"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_23 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

In [ ]:
#test with covariates, categories and strata
outcome     <- "z_24"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_24 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

In [ ]:
#test with covariates, categories and strata
outcome     <- "z_25"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_25 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

In [ ]:
#test with covariates, categories and strata
outcome     <- "z_26"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_26 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

In [ ]:
#test with covariates, categories and strata
outcome     <- "z_27"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_27 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

In [ ]:
#test with covariates, categories and strata
outcome     <- "z_28"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_28 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

In [ ]:
#test with covariates, categories and strata
outcome     <- "z_29"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_29 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

In [ ]:
#test with covariates, categories and strata
outcome     <- "z_30"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_30 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

In [ ]:
#test with covariates, categories and strata
outcome     <- "z_31"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_31 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

In [ ]:
#test with covariates, categories and strata
outcome     <- "z_32"
covariates  <- c("age_at_MRI", "Sex", "Ethnicity", "BSA")  
categories <- as.data.frame(data_dictionary, stringsAsFactors = FALSE)

non_exposure_cols <- c(
  "eid_40616",          
  "z_1", "z_2", "z_3", "z_4", "z_5", "z_6", "z_7", "z_8", "z_9", "z_10", "z_11", "z_12", "z_13", "z_14", "z_15", "z_16", "z_17", "z_18", "z_19", "z_20", "z_21", "z_22", "z_23", "z_24", "z_25", "z_26", "z_27", "z_28", "z_29", "z_30", "z_31", "z_32", 
  "LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global", "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum",
  "age_at_MRI", "Sex", "BSA", "Ethnicity"  
  #"birth_cohort",              
  #"sex", "ethnicity", "crp", "adopted", "age_i0", "BSA_i0"
)

exposures <- setdiff(names(xwas_data), non_exposure_cols)


#outcome    <- "GlycA_trans"
#covariates <- c("BSA_i0", "sex", "age_i0", "age_i0^2", "age_i0:sex", "ethnicity")
#exposures <- setdiff(names(xwas_data), non_exposure_cols)
strata <- c(Cs(Sex))

XWAS_32 <- XWAS(
    data = xwas_data,
    model_type = "lm",
    outcome = outcome,
    #strata = TRUE,
    strata = strata,
    covariates = covariates,
    exposures = exposures,
    #extra.covar = "overall_health",
    #extra.var = "alcohol_freq",
    categories = categories,
    replication = FALSE,
    multi_imputation = FALSE,
    scale = TRUE,
    scale_exclude = factor_vars, 
    p_adjust = "BH",
    cores = NULL,
    interaction = FALSE
) 

In [40]:
write_xlsx(
  list(
      XWAS_z_1 = XWAS_1$summary,
      XWAS_z_2 = XWAS_2$summary,
      XWAS_z_3 = XWAS_3$summary,
      XWAS_z_4 = XWAS_4$summary,
      XWAS_z_5 = XWAS_5$summary,
      XWAS_z_6 = XWAS_6$summary,
      XWAS_z_7 = XWAS_7$summary,
      XWAS_z_8 = XWAS_8$summary,
      XWAS_z_9 = XWAS_9$summary
      
  ),
  path = "exposure_lv_idps.xlsx"
)

In [41]:
system('dx upload exposure_lv_idps.xlsx --path /Mattia/')

In [134]:
## objects: XWAS_1 ... XWAS_32, each with $summary having:
## Exposure, Beta, Beta_2.5%, Beta_97.5%, p.value, FDR, Variable, Category

obj_names <- paste0("XWAS_", 1:32)

cleaned <- map(mget(obj_names), ~ .x$summary %>%
                 mutate(Exposure = sub("\\.L$", "", Exposure)))

for (i in seq_along(cleaned)) {
  assign(obj_names[i], list(summary = cleaned[[i]]))
}

## 1) Build the long table for volcano plot (keeps Category + Variable)
volcano_df <- map2_dfr(
  mget(obj_names), 1:32,
  ~ .x$summary %>%
      transmute(
        Component   = paste0("z_", .y),
        Trait       = Exposure,
        Effect_Size = Beta,
        SE          = NA_real_,         # fill if you have it elsewhere
        CI_L        = `Beta_2.5%`,
        CI_U        = `Beta_97.5%`,
        P_Value     = p.value,
        FDR         = FDR,
        Variable    = Variable,
        Category    = Category
      )
)

## 2) Pivot to wide Beta matrix while keeping Category (and Variable if you want)
betas_matrix <- volcano_df %>%
  select(Trait, Category, Component, Effect_Size) %>%
  pivot_wider(
    id_cols    = c(Trait, Category),
    names_from = Component,
    values_from = Effect_Size
  ) %>%
  arrange(Category, Trait)

## 3) (Optional) If you have a desired row order for traits:
# wanted_traits <- c("alcohol_freqOnce or twice a week", "alcohol_freqThree or four times a week", ...)
# betas_matrix <- betas_matrix %>%
#   mutate(Trait = factor(Trait, levels = wanted_traits)) %>%
#   arrange(Category, Trait) %>%
#   mutate(Trait = as.character(Trait))

## 4) Save both to one Excel file (separate sheets)
write_xlsx(
#  list(
    volcano_df, #  = volcano_df,
#    betas_matrix = betas_matrix
#  ),
  path = "exposure_volcano_df_oct_sex.xlsx"
)
system('dx upload exposure_volcano_df_oct_sex.xlsx --path /Mattia/Motion_genetic/')

In [54]:
# XWAS_1 ... XWAS_9 exist in the environment, each with $summary:
# Exposure, Beta, Beta_2.5%, Beta_97.5%, p.value, FDR, Variable, Category

obj_names  <- paste0("XWAS_", 1:9)
comp_names <- c("LVEF", "LVEDVi", "LVESVi", "Ecc_Global", "Err_Global",
                "Ell_Global", "PDSR_longit", "PDSR_radial", "PDSR_circum")

stopifnot(length(obj_names) == length(comp_names))

## --- 0) Load and clean summaries (remove trailing ".L" from Exposure) -----
summaries_clean <- mget(obj_names) %>%
  map(~ .x$summary %>%
        mutate(Exposure = sub("\\.L$", "", Exposure)))

# (Optional) reassign cleaned summaries back to your XWAS_* objects:
for (i in seq_along(summaries_clean)) {
  assign(obj_names[i], list(summary = summaries_clean[[i]]))
}

## --- 1) Build LONG table for volcano plot (keeps Variable + Category) -----
volcano_idps_exopo_df <- map2_dfr(
  summaries_clean, comp_names,
  ~ .x %>%
      transmute(
        Component   = .y,              # use your component names
        Trait       = Exposure,
        Effect_Size = Beta,
        SE          = NA_real_,        # fill if available
        CI_L        = `Beta_2.5%`,
        CI_U        = `Beta_97.5%`,
        P_Value     = p.value,
        FDR         = FDR,
        Variable    = Variable,
        Category    = Category
      )
)

## --- 2) Wide beta matrix: rows = (Trait, Category), cols = components -----
betas_matrix <- volcano_idps_exopo_df %>%
  select(Trait, Category, Component, Effect_Size) %>%
  pivot_wider(
    id_cols     = c(Trait, Category),
    names_from  = Component,
    values_from = Effect_Size
  ) %>%
  arrange(Category, Trait)

## --- 3) (Optional) Force Trait to be treated as text in Excel -------------
# This prevents Excel from misinterpreting strings with < or >.
# Uncomment if you want the safeguard:
# volcano_df   <- volcano_df   %>% mutate(Trait = paste0("'", Trait))
# betas_matrix <- betas_matrix %>% mutate(Trait = paste0("'", Trait))

## --- 4) Save to Excel -----------------------------------------------------
# a) If you only want the volcano_df (like your previous script):
write_xlsx(volcano_idps_exopo_df, path = "exposure_lv_idps_volcano_df.xlsx")

# b) Or save both long and wide to one workbook:
write_xlsx(
  volcano_idps_exopo_df,
  path = "exposure_lv_idps_volcano.xlsx"
)

## --- 5) (Optional) Keep exporting the raw sheets as you already do --------
write_xlsx(
  list(
    XWAS_LVEF         = XWAS_1$summary,
    XWAS_LVEDVi       = XWAS_2$summary,
    XWAS_LVESVi       = XWAS_3$summary,
    XWAS_Ecc_Global   = XWAS_4$summary,
    XWAS_Err_Global   = XWAS_5$summary,
    XWAS_Ell_Global   = XWAS_6$summary,
    XWAS_PDSR_longit  = XWAS_7$summary,
    XWAS_PDSR_radial  = XWAS_8$summary,
    XWAS_PDSR_circum  = XWAS_9$summary
  ),
  path = "exposure_lv_idps_summaries.xlsx"
)


In [55]:
system('dx upload exposure_lv_idps_volcano.xlsx --path /Mattia/')
system('dx upload exposure_lv_idps_summaries.xlsx --path /Mattia/')